In [1]:
import pandas as pd
from song_orderer import Playlist

In [2]:
tekno_drums_thursday_df = Playlist('https://open.spotify.com/playlist/4h1y1dG7hlMKAbwiT8fL3h?si=927fa37877214b5b').get_playlist_data()

In [3]:
tekno_drums_thursday_df

,Track Name,Artist Name,Album Name,Release Date,Duration (ms),Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Valence,Tempo,Key,Mode,Time Signature,Genres,Top Genre
0,Transdimensional,TAAHLIAH,Angelica,2021-05-28,289865,30,0.042500,0.797,0.907,0.890000,0.1060,-5.389,0.1100,0.4460,150.009,7,1,4,"{deconstructed club, pink noise}",deconstructed club
1,Digital Requiem,Mercury 200,Supply And Demand,2023-04-14,376666,15,0.001670,0.619,0.980,0.917000,0.1420,-6.864,0.0505,0.8020,145.006,1,1,4,{techno rave},techno rave
2,Baby Bliss,Europa,Heart of Brass,2021-01-17,339513,24,0.021000,0.511,0.820,0.509000,0.3490,-9.389,0.0375,0.2920,164.001,10,0,4,"{hypertrance, bubblegum bass}",hypertrance
3,Silkworm,Giant Swan,Silkworm,2020-11-26,325590,25,0.265000,0.771,0.756,0.870000,0.0988,-9.685,0.0830,0.5380,136.007,1,1,3,"{bristol electronic, bristol indie, deconstruc...",bristol electronic
4,Outrenoir,RAVL,Nü Kvlture VOL. 3,2023-01-03,309000,17,0.089400,0.604,0.995,0.885000,0.0589,-2.712,0.1220,0.3430,150.019,10,1,4,{},None
5,HOT,Manni Dee,HOT,2023-06-06,211948,22,0.000009,0.596,0.928,0.927000,0.2080,-6.936,0.0430,0.3060,145.005,1,1,4,"{dark techno, minimal dub, raw techno}",dark techno
6,Speedo,VTSS,ALVIN,2020-11-13,374703,13,0.000139,0.738,0.905,0.882000,0.0938,-8.203,0.0464,0.1790,151.991,1,1,4,"{deconstructed club, polish techno, raw techno}",deconstructed club
7,bbbase,Buttechno,badtrip,2023-05-18,370344,16,0.001490,0.877,0.716,0.915000,0.1040,-6.779,0.0845,0.8170,124.832,9,1,4,"{russian electronic, russian techno}",russian electronic
8,Flume,Gyru Machina,Gouryello,2022-07-25,63360,40,0.333000,0.777,0.715,0.677000,0.0835,-13.646,0.0672,0.0747,140.015,11,0,4,{},None
9,Ride - Dagga Remix,Jensen Interceptor,Ride (Dagga Remix),2022-02-25,295711,29,0.000119,0.609,0.951,0.826000,0.1400,-7.008,0.0503,0.2900,145.016,4,0,4,"{ghettotech, destroy techno, australian techno}",ghettotech
